In [62]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [63]:
import unicodedata, re      # re: 해쉬태그에 대한 정규 표현식
from tqdm.notebook import tqdm

In [64]:
insta_url = 'https://www.instagram.com/'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(insta_url)
time.sleep(1)

### 1. 로그인

In [65]:
with open('password.txt') as f:
    password = f.read()

In [66]:
email = 'bluemilktea0'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [67]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)

In [68]:
input_pwd.submit()
time.sleep(1)

In [69]:
# 로그인 정보 저장 - 나중에 하기
# 에러나도 다음으로 넘어가게

try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [70]:
# 알림설정 - 나중에 하기

try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주맛집으로 검색

In [71]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [72]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글

In [73]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [74]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [75]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 752🔸영업 : 오전11시~오후10시🔸전화: 064-743-1979🔸연중무휴 @lobstervin#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집#제주카페 #외도맛집 #하귀맛집#한림맛집 #한림카페 #애월해안도로맛집#애월가볼만한곳 #제주공항맛집 #제주데이트 #제주공항근처맛집 #한담맛집 #제주해안도로#제주맛집추천 #제주도맛집추천 #제주도민맛집'

In [76]:
# 본문에서 해쉬태그 가져오기, '#[^\s#,\\]+'

tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#애월맛집',
 '#애월카페',
 '#애월읍맛집',
 '#제주애월맛집',
 '#제주도맛집',
 '#제주맛집',
 '#제주카페',
 '#외도맛집',
 '#하귀맛집',
 '#한림맛집',
 '#한림카페',
 '#애월해안도로맛집',
 '#애월가볼만한곳',
 '#제주공항맛집',
 '#제주데이트',
 '#제주공항근처맛집',
 '#한담맛집',
 '#제주해안도로',
 '#제주맛집추천',
 '#제주도맛집추천',
 '#제주도민맛집']

In [77]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.RhOlS.Nzb55')['datetime'][:10]
date

'2021-12-06'

In [78]:
# 4) 좋아요 수 가져오기

try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [79]:
# 5) 위치정보 가져오기

try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [80]:
row = [content, date, like, place, tags]

In [81]:
# 첫번째 게시글에서 다음 게시글로 이동

driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(1)

In [82]:
# 두번째 게시글에서 다음 게시글로 이동

driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(1)

In [83]:
# 5를 하기위해 팝업화면을 없앰 (X클릭)

driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(1)

### 5. 반복문으로 데이터 가져오기

In [84]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#,\\]+', content) 
    date = soup.select_one('time.FH9sR.RhOlS.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0

    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [85]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [86]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

In [87]:
driver.close()

### 6. 데이터 프레임으로 만들고 중복제거

In [90]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 7...,2021-12-06,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
1,💕순천미향 제주산방산본점오늘도 정상 영업 중입니다🤗제주제왕삼합 비쥬얼 미쳤‼️비쥬얼...,2021-12-07,0,순천미향,[]
2,#제주맛집산도롱맨도롱----------🍜갈비국수 대표맛집 🍜📌 산도롱의 대표메뉴 홍...,2021-12-08,0,산도롱맨도롱,"[#제주맛집산도롱맨도롱----------🍜갈비국수, #성산일출봉맛집.대기시간이, #..."
3,🎀 제주에 진짜 맛집 🎀⁣⁣문개항아리 (づ￣ 3￣)づ⁣⁣⁣직접 잡아오는 제주도 자연...,2021-12-07,0,,"[#제주맛집, #문어라면, #애월해물라면, #애월해산물맛집, #제주애월맛집추천, #..."
4,"제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집""성산을본갈치""성산...",2021-12-07,0,,[#성산을본갈치주소]


In [91]:
df.drop_duplicates(subset='content', keep='first', inplace=True)
df.shape

(322, 5)

In [92]:
df.to_csv('인스타-제주맛집.csv', index=False)